# Designing a Chlorination Unit <a class="anchor" id="top"></a>

- **Prepared by:**
    
    - [Yalin Li](https://qsdsan.readthedocs.io/en/latest/authors/Yalin_Li.html)
    - [Eva Reynaert](https://www.eawag.ch/en/aboutus/portrait/organisation/staff/profile/eva-reynaert/show/)
    - [Philipp Steiner](https://www.eawag.ch/en/aboutus/portrait/organisation/staff/profile/philipp-steiner/show/)

- **Covered topics:**

    - [1. Design Algorithms](#s1)
    - [2. Process Algorithms](#s2)
    - [3. Unit Classes](#s3)

---
### Note
This tutorial is under active development.

---

In [ ]:
import qsdsan as qs
print(f'This tutorial was made with qsdsan v{qs.__version__}.')

### Summary
In this example, we will show how we can set up a chlorination process in `QSDsan`, which would include a contact zone, mixing/storage tanks for the chemical sodium hypochlorite (NaOCl) and treated water, and pumps (contact zone, NaOCl dosing, water storage).

## 1. Design Algorithms <a class="anchor" id="s1"></a>

### 1.1. Contact zone

In the contact zone, chlorine (in the form of NaOCl) is added and reacts with the influent stream to inactivate microorganims (e.g., viruses, bacteria, protoza). Two configurations - serpentine tubing and cylindrical tank - are considered here.

**Q1:** Below it only outlines method to calculate CT, we'll still need either a C or a T, so I assume the user will provide the T

#### 1.1.1. Serpentine tubing

For the serpentine tubing configuration (continuous), the tubing will be designed based on the baffling factor, pipe length-to-diameter ratio (aspect ratio), and the desired contact time.

Specifically,

$$
T_{DT} [min] = \frac{T_{contact}}{BF} = \frac{L_p}{v} = L_p * \frac{\pi*{(\frac{d_p}{2})^2}}{Q} = (AS*d_p) * \frac{\pi*{(\frac{d_p}{2})^2}}{Q}
$$

where:
- $T_{DT}$ is the theoretical detention time
- $T_{contact}$ is the desired contact time
- BF is the baffling factor, 0.7 for the serpentine tubing configuration
- $L_p$ and $d_p$ are the length and diameter of the pipe, respectively
- AS is the aspect ratio as in $\frac{L_p}{d_p}$, recommended to be ≧160 by the Colorado Department of Public Health and Environment as in page 16 of this [Baffling Factor Guidance Manual](https://www.colorado.gov/pacific/sites/default/files/CDPHE%20Baffling%20Factor%20Guidance%20Manual.pdf)
- Q and v are the volumetric flow rate and velocity of the influent stream, respectively

solve for $d_p$:

$$
d_p [m] = (\frac{4T_{DT}*Q}{\pi*AS})^{1/3}
$$

Then we can calculate the amount of material needed:

$$
V_{PVC} [m^3] = \pi * L_p * ((\frac{d_p}{2}+t_{pipe})^2 - d_p^2)
$$

where $t_{pipe}$ is the thickness of the pipe.

**Q2:** The original equation has $\frac{d_p+t_{pipe}}{2}$ which I think is inaccurate, so I changed it to ($\frac{d_p}{2}+t_{pipe})$

**Q3:** Additionally, in choosing the tubings, as the size of the tubes are mostly at fixed intervals, so I'd suggest looking into what the actual sizes of the tubes are (e.g., I did a quick search and found this [here](https://pvcpipesupplies.com/pvc-pipe), but it's a U.S. site), which can also provide the cost data

#### 1.1.2. Cylindrical tank

For the cylindrical tank configuration (batch), the volume is calculated based on the refill interval (set to 7 days), tank height-to-diameter ratio (set to 2), and the total tank-to-chlorine solution ratio

**Q4:** I'm confused by how the cylindrical tank volume is calcualted - from the slide it's merely based on the volume of the NaOCl solution, but we should also include the influent stream

I think we should probably use an approach similar to the serpentine tubing above (e.g., assume a retention time, baffling factor).

However, it also occurs to me that the equations in the slide is similar to what we'll use to calculate the size of the storage of NaOCl solution, so can you also clarify what this tank is used for?

The total PVC volume is:

$$
V_{PVC} [m^3] = V_{wall} + V_{floor}
$$

$$
V_{wall} = \pi*h_{cyl}*((d_{cyl}+2*t_{cyl})^2-d_{cyl}^2) = \pi*AS*d_{cyl}*((d_{cyl}+2*t_{cyl})^2-d_{cyl}^2)
$$

$$
V_{floor} = \pi*(d_{cyl}+2*t_{cyl})^2*t_{cyl}
$$

where:
- $h_{cyl}$, $d_{cyl}$, and $t_{cyl}$ are the height, inner diameter, and wall thickness of the cylindrical tank, respectively
- AS is the aspect ratio as in $\frac{h_cyl}{d_cyl}$

**Q5:** In calculating $M_{cyl}$, the equation in the slide uses $\frac{d_{cyl}}{2}$ to represent $h_{cyl}$, I think it should be $2d_{cyl}$ (as in the equation I wrote above if the height-to-diameter ratio is 2?

**Q6:** Additionally, I think the thickness of the tank should be larger than the one assumed for the tubings, I also realized that our generic algorithms for tank design are mostly for concrete/stainless steel tanks, so we'll need assumptions/literature to determine the dimensions of the tanks.

### 1.2. Pumps

For the design of the pumps, we will use the general algorithms in the `WWTpump` class in `QSDsan` (despite of the name, the pump algorithms are not limited to wastewater treatment settings).

### 1.3. Tanks

For mixing/storage thanks, we will calculate the volume based on the flow rate, retention time, and a safety factor. Power needed for the mixing tank will be based on the volume of the tank.

## 2. Process Algorithms <a class="anchor" id="s2"></a>

### 2.1. Chlorine dose

To determine the amount of NaOCl to be added, we will need to calculate the CT (concentration$*$time) values required by the inactivation target.

Let's assume that we will use the following table from U.S. Environmental Protection Agency to determine the CT (in min-mg/L) for 4-log inactivation of viruses by free chlorine (Table B-2 on Page B-3 in this [Disinfection Profiling and Benchmarking Technical Guidance Manual](https://www.epa.gov/system/files/documents/2022-02/disprof_bench_3rules_final_508.pdf)).

| Temperature (°C) | pH=6-9 | pH=10 |
|       :-:        |   :-:  |  :-:  |
|       0.5        |   12   |   90  |
|         5        |    8   |   60  |
|        10        |    6   |   45  |
|        15        |    4   |   30  |
|        20        |    3   |   22  |
|        25        |    2   |   15  |

**Q7:** I *think* the table in the slide is the old one (maybe from [here](https://www.google.com/books/edition/Disinfection_Profiling_and_Benchmarking/F5AehDWyEa8C?hl=en&gbpv=1&dq=table+3-5.+required+ct+values+(mg-min/L)+for+4-log+inactivation+of+viruses+by+free+chlorine,+ph+6.0-9.0&pg=SA3-PA20&printsec=frontcover)?), so I used this newer table instead, feel free to change back

With a given contact time, we can calculate the residual free chlorine concentration $C_{res}$:

$$
C_{res} [\frac{mg}{L}] = \frac{CT}{C_{res}}
$$

Based on the following equation to take into account the amount of chlorine lost to reactions with organics (quantified as the total organic carbon, TOC) and UVA, we can back-calculate $C_0$ using $C_{res}$:

$$
C_{res} = -0.8404C_0*ln\frac{C_0}{C_{res}} - 0.404TOC*T_{contact}*(\frac{C_0}{UVA})^{-0.9108} + C_0
$$

**Q8:** I'm not sure how TOC and UVA are quantified (e.g., units for them in the equation above)?

With $C_0$ solved, we will know how much NaOCl we need to add to achieve the desired CT.

### 2.2. Pumping energy

Pumping energy can be calculated based on the flow rate and head pressure/loss as:

$$
P [kW] = \frac{mgH}{1000\eta}
$$

where:
- $m$ is mass flow rate in $[\frac{kg}{s}]$
- $H$ is the head pressure/loss $[m]$
- $\eta$ is the typical pump efficiency (set to 60%)

#### 2.2.1. For chlorine addition

To get the chlorine feed pump power, mass flow rate of chlorine can be calculated from the chlorine dose $C_0$ and Q) and the head pressure H is assumed to be 70.3 m.

#### 2.2.2. For contact zone

**Serpentine tubing**

In the case of serpentine tubing, head loss is the sum of the major head loss ($H_f$; due to friction) and minor head loss ($H_m$; due to bends in flow):

$$
H [m] = H_f + H_m
$$

For the major head loss, the [Hazen-Williams equation](https://en.wikipedia.org/wiki/Hazen%E2%80%93Williams_equation) can be used:

$$
H_f = \frac{0.2083*(\frac{100*Q}{C})^{1.852}}{100*d_p^{4.8655}} * L_p
$$

where C is the roughness coefficient and assumed to be 150 for PVC.

**Q9:** Do we have a citation for this equation (I only found the general one from Wikipedia, but the values of the exponents seem very specific

The minor head loss can be calculated as:

$$
H_m = \frac{\epsilon*v^2}{2g} * N_{bend}
$$

where:
- $\epsilon$ is the minor loss coefficient and assumed to be 1.5
- $N_{bend}$ is the number of bends can be calculated by dividing the total length by the segment length

$N_{bend}$ can be calculated as

$$
N_{bend} = \frac{L_p}{L_{seg}}
$$

and the segment length $L_{seg}$ can be calculated based on the segment length-to-diameter ratio (recommended to be ≦40 by the Colorado Department of Public Health and Environment as in page 15 of this [Baffling Factor Guidance Manual](https://www.colorado.gov/pacific/sites/default/files/CDPHE%20Baffling%20Factor%20Guidance%20Manual.pdf)).

**Cylindrical tank**

For cylindrical tank, there is no minor head loss, therefore the total head loss only comes from the friction loss:

$$
H = H_f = \frac{0.2083*(\frac{100*Q}{C})^{1.852}}{100*d_{cyl}^{4.8655}} * h_{cyl}
$$

[Back to top](#top)

## 3. Unit Classes <a class="anchor" id="#s3"></a>

### 3.1. Contact zone

For the contact zone, we need to create a new class. Check out the tutorials on `SanUnit` ([basic](https://qsdsan.readthedocs.io/en/latest/tutorials/4_SanUnit_basic.html), [advanced](https://qsdsan.readthedocs.io/en/latest/tutorials/5_SanUnit_advanced.html)) for how to make a new `SanUnit` subclass.

In [ ]:
from math import log, pi, ceil
from flexsolve import IQ_interpolation
from qsdsan import SanUnit, Construction
from qsdsan.sanunits import WWTpump

class ContactZone(SanUnit):
    '''
    Contact zone for water disinfection using chlorine (added in the form of sodium hypochlorite, NaOCl).
    
    Parameters
    ----------
    ins : Iterable(obj)
        Influent stream, NaOCl (updated in upon unit simulation).
    outs : obj
        Disinfected stream.
    configuration : str
        Can be either 'serpentine_tubing' or 'cylindrical_tank'.
    target_CT : float
        Desired CT (concentration*time) for microorganism in min-mg/L.
    contact_time : float
        Desired contact time in min.
    UVA : float
        Disinfection credit from UVA.
    PVC_thickness : float
        Thickness of the PVC material in m.
    material_unit_costs : dict(str, float)
        Unit cost of the materials in their functional units.
        
    Examples
    --------
    Here we will skip this as we will show how to use it later.
    '''
    _N_ins = 2 # influent stream, NaOCl solution
    _N_outs = 1 # disinfected water
    _baffling_factor = {
        'serpentine_tubing': 0.7,
        'cylindrical_tank': 0.1,
    }
    _aspect_ratio = {
        'serpentine_tubing': 160,
        'cylindrical_tank': 2,
    }
    segment_L_to_dia = 40 # segment length to diameter ratio for serpentine tubing
    C = 150 # roughness coefficient
    epsilon = 1.5 # minor loss coefficient
    pump_eff = 0.6
    
    def __init__(self, ID='', ins=None, outs=(), thermo=None, init_with='WasteStream',
                 configuration='serpentine_tubing',
                 target_CT=4, # based on the table, set default at T=15°C and pH=6-9
                 contact_time=10, UVA=1, PVC_thickness=0.005,
                 material_unit_costs={ # all made up now
                     'PVC': 10,
                     'StainlessSteel': 10,
                 },
                 **kwargs):
        SanUnit.__init__(self, ID, ins, outs, thermo, init_with)
        self.contact_time = contact_time
        self.configuration = configuration
        self.target_CT = target_CT
        self.UVA = UVA
        self.PVC_thickness = PVC_thickness
        for attr, val in kwargs:
            setattr(self, kwargs)
        
        # To consider LCA impacts from the construction material
        self.construction = (
            Construction('contact_zone_PVC', linked_unit=self,
                         item='PVC', quantity_unit='kg',
                         price=material_unit_costs['PVC']),
            Construction('contact_zone_SS', linked_unit=self,
                         item='StainlessSteel', quantity_unit='kg',
                         price=material_unit_costs['StainlessSteel']),
        )
        
        # Pump
        ID = self.ID
        mixed = self._mixed = self.ins[0].copy(ID+'_mixed')
        self.pump = WWTpump(
            ID=ID+'_pump', ins=mixed.proxy(mixed.ID+'_proxy'),
            pump_type='', # use the generic pump algorithm
            N_pump=1, capacity_factor=1, include_pump_cost=True,
            include_building_cost=False, include_OM_cost=False,
        )
        
    # Target function to solve C_0
    @staticmethod
    def _C_res_at_C_0(C_0, TOC, contact_time, UVA, C_res):
        C_res2 = -0.8404*C_0*log(C_0/C_res) - 0.404*TOC*contact_time*(C_0/UVA)^(-0.9108) + C_0
        return C_res2-C_res
        
    # Implement process algorithms
    def _run(self):
        inf, naocl = self.ins
        eff, = self.outs
        
        # Calculate C_res
        TOC = inf.TOC # in mg/L
        contact_time = self.contact_time
        C_res = self.target_CT / contact_time # in mg/L
        C_0 = IQ_interpolation( # in mg/L
            f=self._C_res_at_C_0, x0=C_res, x1=100*C_res, # assume that C_0 won't be >100X of C_res
            xtol=1e-3, ytol=1e-6, args=(TCO, contact_time, UVA, C_res),
            checkbounds=False)
        naocl.empty()
        C_naocl = C_0/70.91*74.44 # 1-to-1 molar conversion of C_0 (for Cl2) to NaOCl
        naocl.imass['NaOCl'] = m_naocl = inf.F_vol * C_naocl / 1000 # 1000*m3*mg/L = kg
        naocl.imass['H2O'] = m_naocl/0.15-m_naocl # 15 wt/wt% NaOCl solution
        
        eff.mix_from((inf, naocl))
        eff.imass['NaOCl'] *= C_res/C_0 # account for the consumed NaOCl
        
    _units = { # units of measure for the design parameters
        'Pipe diameter': 'm',
        'Pipe length': 'm',
        'Tank diameter': 'm',
        'Tank height': 'm',
        'Total PVC': 'm3',
        'Pump head': 'm',
        'Pump stainless steel': 'kg',
    }
        
    # Implement design algorithms
    def _design(self):
        D = self.design_results
        D.clear() # in case that the configuration is changed
        # Inputs needed by both configurations
        t_dt = self.contact_time / self.baffling_factor # theoretical detention time
        Q = self.F_vol_in # m3/hr
        t_PVC, AS, C = self.PVC_thickness, self.aspect_ratio, self.C
        dia = (4*t_dt*Q/(pi*AS))^(1/3)
        dia_out = dia + 2*t_PVC
        if self.configuration == 'serpentine_tubing':
            # PVC reactor
            D['Pipe diameter'] = dia
            L_p = D['Pipe length'] = dia * AS
            V_PVC = D['Total PVC'] = pi * L_p * ((dia_out/2)^2-dia^2)
            # Pump head
            H_f = 0.2083*(100*Q/C)^1.852/(100*dia^4.8655)*L_p
            v = Q/(pi*dia^2)
            N_bend = ceil(L_p/(dia*self.segment_L_to_dia))
            H_m = self.eta*v^2/(2*9.81)*N_bend
            H = D['Pump head'] = H_f + H_m
        else:
            # PVC reactor
            D['Tank diameter'] = dia
            h_cyl = D['Tank height'] = dia * AS
            V_wall = pi*h_cyl*(dia_out^2-dia^2)
            V_floor = pi * dia_out^2 * t_PVC
            V_PVC = D['Total PVC'] = V_wall + V_floor
            # Pump head
            H = H_f = 0.2083*(100*Q/C)^1.852/(100*dia^4.8655)*h_cyl

        # Pump
        self.mixed.mix_from(self.ins)
        pump = self.pump
        pump.simulate()
        m_ss = D['Pump stainless steel'] = pump.design_results['Pump stainless steel']
        self.power_utility.rate = self.F_mass_in*9.81*H/(1000*self.pump_eff)
        
        # Construction materials for TEA/LCA
        self.construction[0].quantity = V_PVC
        self.construction[1].quantity = m_ss
        self.add_construction(add_cost=True)
        
    _F_BM_default = {
        'contact_zone_PVC': 1,
        'contact_zone_SS': 1,
        'Pump': 1.18*(1+0.007/100),
    }
    def _cost(self):
        C = self.baseline_purchase_costs
        C['Pump'] = self.pump.baseline_purchase_costs['Pump']
            
    # Add these as properties to include checks/flexibility
    @property
    def configuration(self):
        '''Configuration of the contact zone, can be either "serpentine_tubing" or "cylindrical tank".'''
        return self._configuration
    @configuration.setter
    def configuration(self, config):
        lower = config.lower()
        if lower in ('serpentine_tubing', 'cylindrical_tank'):
            self._configuration = lower
        else:
            raise ValueError('`configuration` can only be "serpentine_tubing" or "cylindrical_tank", '
                             f'not "{config}".')
        
    @property
    def baffling_factor(self):
        '''Baffling factor for the configuration, should be between 0 and 1 where 1 is perfect (plug flow).'''
        return self._baffling_factor[self.configuration]
    @baffling_factor.setter
    def baffling_factor(self, BF):
        if 0 < BF <= 1:
            self._baffling_factor[self.configuration] = BF
        else:
            raise ValueError(f'`baffling_factor` should be within (0, 1], not "{BF}".')
            
    @property
    def aspect_ratio(self):
        '''Pipe length over diameter (serpentine piping) or tank height over diameter (cylindrical tank).'''
        return self._aspect_ratio[self.configuration]
    @aspect_ratio.setter
    def aspect_ratio(self, AS):
        self._aspect_ratio[self.configuration] = AS

### 3.2. ChlorineMixing

For the chlorine addition/mixing tank, we will use the general algorithms of a `MixTank`, but add an auxiliary pump.

**Q10:** What is the assumed lifetime of the materials and the lifetime of the TEA/LCA?

In [ ]:
from qsdsan.sanunits import MixTank

class ChlorineMixing(MixTank):
    '''
    A subclass of `MixTank` with an auxiliary pump.
    
    Parameters
    ----------
    ins : Iterable(obj)
        Influent streams to be mixed.
    outs : obj
        Disinfected stream.
    head_pressure : float
        Assumed head pressure for the pump.
        
    See Also
    --------
    `qsdsan.sanunits.MixTank <https://qsdsan.readthedocs.io/en/latest/sanunits/tanks.html#mixtank>`_
    '''
    def __init__(self, ID='', ins=None, outs=(), thermo=None, init_with='WasteStream',
                 head_pressure=70.3, **kwargs):
        super().__init__(self, ID, ins, outs, thermo, init_with, **kwargs)
        self.head_pressure = head_pressure
        eff = self.outs[0]
        self.pump = WWTpump(
            ID=self.ID+'_pump', ins=eff.proxy(eff.ID+'_proxy'),
            pump_type='', # use the generic pump algorithm
            N_pump=1, capacity_factor=1, include_pump_cost=True,
            include_building_cost=False, include_OM_cost=False,
        )
        self._units['Pump stainless steel'] = 'kg'
        self._BM['Pump'] = 1.18*(1+0.007/100)
    
    def _design(self):
        super()._design()
        D = self.design_results
        # Pump
        pump = self.pump
        pump.simulate()
        m_ss = D['Pump stainless steel'] = pump.design_results['Pump stainless steel']

    def _cost(self):
        pump = self.pump
        self.baseline_purchase_costs['Pump'] = pump.baseline_purchase_costs['Pump']
        self.power_utility.rate = pump.power_utility.rate

## 4. Process System

Finally it's time to create and simulate the entire system.

In [ ]:
# Identify the components needed for simulation
from qsdsan import Component, Components, set_thermo
kwargs = {
    'phase': 'l',
    'particle_size': 'Soluble',
    'degradability': 'Undegradable',
    'organic': False,
}
H2O = Component('H2O', **kwargs)
NaOCl = Component('NaOCl', **kwargs)
HCl = Component('HCl', **kwargs)
HOCl = Component('HOCl', **kwargs)
NH3 = Component('NH3', **kwargs) # assumed to be liquefied NH3
cmps = Components([H2O, NaOCl, HCl, HOCl, NH3])
cmps.default_compile() # check the documentation for the assumptions behind `default_compile`
cmps.set_alias('H2O', 'Water')
cmps.set_alias('NH3', 'Ammonia')
set_thermo(cmps)

[Back to top](#top)